In [217]:
import numerical
reload(numerical)

fun = lambda x: (3 * x[0] + x[1]) ** 2

print numerical.hess(fun)([3, 1])

[[ 18.00160021   6.0005334 ]
 [  6.0005334    2.00373051]]


In [215]:
# http://stackoverflow.com/questions/35266370/tensorflow-compute-hessian-matrix-and-higher-order-derivatives
import tensorflow as tf      

def hessian(fn, vars):
    cons = lambda x: tf.constant(x, dtype=tf.float32)
    mat = []
    for v1 in vars:
        temp = []
        for v2 in vars:
            # computing derivative twice, first w.r.t v2 and then w.r.t v1
            temp.append(tf.gradients(tf.gradients(f, v2)[0], v1)[0])
        # tensorflow returns None when there is no gradient, so we replace None with 0
        temp = [cons(0) if t == None else t for t in temp] 
        temp = tf.pack(temp)
        mat.append(temp)
    mat = tf.pack(mat)
    return mat

x = tf.Variable(0, dtype = tf.float32)
y = tf.Variable(0, dtype = tf.float32)
f = (3 * x + y) ** 2

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print hessian(f, [x, y]).eval()

[[ 18.   6.]
 [  6.   2.]]


In [211]:
import theano
import theano.tensor as T

x = T.dvector('x')
y = x ** 2
cost = y.sum()
gy = T.grad(cost, x)
H, updates = theano.scan(lambda i, gy,x : T.grad(gy[i], x), sequences=T.arange(gy.shape[0]), non_sequences=[gy, x])
f = theano.function([x], H, updates=updates)